In [1]:
import pandas as pd
import numpy as np
import io
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
from google.colab import files
uploaded = files.upload()

Saving global_warming_dataset.csv to global_warming_dataset.csv


In [3]:
gw_df = pd.read_csv(io.BytesIO(uploaded['global_warming_dataset.csv']))
print(gw_df.head())

# Data Cleaning: Drop duplicates and handle missing values
gw_df = gw_df.drop_duplicates().dropna()

       Country  Year  Temperature_Anomaly  CO2_Emissions    Population  \
0  Country_103  1913            -1.163537   8.876061e+08  1.627978e+08   
1  Country_180  1950            -0.432122   4.497517e+08  4.281359e+08   
2   Country_93  2014             0.444954   4.579080e+08  4.926732e+08   
3   Country_15  2020            -1.171616   5.049503e+08  1.252169e+09   
4  Country_107  1964            -0.564038   6.898891e+08  2.932960e+08   

   Forest_Area           GDP  Renewable_Energy_Usage  Methane_Emissions  \
0    54.872178  6.139887e+12               76.710013       8.317626e+06   
1    84.051006  2.601447e+12               68.450021       6.206540e+06   
2    72.295357  5.192677e+12               36.725699       1.056885e+06   
3    17.259684  8.252128e+12               77.547901       1.986813e+06   
4    44.438605  8.560746e+12               10.019576       3.313252e+06   

   Sea_Level_Rise  ...  Waste_Management  Per_Capita_Emissions  \
0        8.111839  ...         82.6914

In [5]:
# Feature Selection and Normalization
# Select only numerical features for scaling
numerical_features = gw_df.select_dtypes(include=np.number).columns.tolist()  # Get list of numerical columns
numerical_features.remove('CO2_Emissions')  # Remove the target variable
X = gw_df[numerical_features]  # Features
y = gw_df['CO2_Emissions']  # Target variable

# Normalize numerical features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Standardize the features
std_scaler = StandardScaler()
X_standardized = std_scaler.fit_transform(X_scaled)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_standardized, y, test_size=0.2, random_state=42)

# Build the TensorFlow model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Model Evaluation:\nMAE: {mae}\nMSE: {mse}\nRMSE: {rmse}\nR2 Score: {r2}")


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5000/5000 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 332596082649333760.0000 - mae: 499439744.0000 - val_loss: 334486452375126016.0000 - val_mae: 501465536.0000
Epoch 2/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 332587492714741760.0000 - mae: 499406912.0000 - val_loss: 331040239336292352.0000 - val_mae: 498033856.0000
Epoch 3/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 328334822256672768.0000 - mae: 494785472.0000 - val_loss: 323815726387298304.0000 - val_mae: 490841248.0000
Epoch 4/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 318711484173516800.0000 - mae: 484904480.0000 - val_loss: 311909630366187520.0000 - val_mae: 478996384.0000
Epoch 5/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 305606748679438336.0000 - mae: 472539648.0000 - val_loss: 294733506393669632.0000 - val_mae: 461891904.0000
Epoch 6/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 287830463156846592.0000 - mae: 454355168.0000 - val_loss: 272070785038811136.0000 - val_mae: 